In [5]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('chinook.db')

customers = pd.read_sql_query("SELECT * FROM customers", conn)
invoices = pd.read_sql_query("SELECT * FROM invoices", conn)
invoice_items = pd.read_sql_query("SELECT * FROM invoice_items", conn)
tracks = pd.read_sql_query("SELECT * FROM tracks", conn)

# Customer Purchases Analysis
customer_spending = customers.merge(invoices, on='CustomerId', how='left')
customer_spending['CustomerName'] = customer_spending['FirstName'] + ' ' + customer_spending['LastName']
customer_spending = (customer_spending.groupby(['CustomerId', 'CustomerName'])['Total']
                    .sum()
                    .round(2)
                    .reset_index()
                    .sort_values(by='Total', ascending=False)
                    .head(5))

print("Top 5 Customers by Total Purchase Amount:")
print(customer_spending[['CustomerId', 'CustomerName', 'Total']])
print("\n")

# Album vs. Individual Track Purchases
album_tracks = tracks.groupby('AlbumId')['TrackId'].count().reset_index()
album_tracks.columns = ['AlbumId', 'TotalTracks']

purchase_data = (invoice_items.merge(invoices, on='InvoiceId')
                            .merge(tracks[['TrackId', 'AlbumId']], on='TrackId')
                            .merge(customers[['CustomerId']], on='CustomerId'))

tracks_purchased = (purchase_data.groupby(['CustomerId', 'AlbumId'])['TrackId']
                   .nunique()
                   .reset_index()
                   .rename(columns={'TrackId': 'TracksPurchased'}))

purchase_analysis = tracks_purchased.merge(album_tracks, on='AlbumId')

purchase_analysis['PurchaseType'] = purchase_analysis.apply(
    lambda x: 'Individual Tracks' if x['TracksPurchased'] < x['TotalTracks'] else 'Full Album', axis=1)

customer_preference = (purchase_analysis.groupby('CustomerId')['PurchaseType']
                      .agg(lambda x: 'Individual Tracks' if 'Individual Tracks' in x.values else 'Full Album')
                      .reset_index()
                      .rename(columns={'PurchaseType': 'Preference'}))

# Calculate percentages
total_customers = len(customer_preference)
individual_track_customers = len(customer_preference[customer_preference['Preference'] == 'Individual Tracks'])
full_album_customers = len(customer_preference[customer_preference['Preference'] == 'Full Album'])

individual_percentage = (individual_track_customers / total_customers) * 100
full_album_percentage = (full_album_customers / total_customers) * 100

print("Customer Purchase Preference Summary:")
print(f"Percentage of customers preferring Individual Tracks: {individual_percentage:.2f}%")
print(f"Percentage of customers preferring Full Albums: {full_album_percentage:.2f}%")

conn.close()

Top 5 Customers by Total Purchase Amount:
    CustomerId        CustomerName  Total
5            6         Helena Holý  49.62
25          26  Richard Cunningham  47.62
56          57          Luis Rojas  46.62
44          45     Ladislav Kovács  45.62
45          46       Hugh O'Reilly  45.62


Customer Purchase Preference Summary:
Percentage of customers preferring Individual Tracks: 100.00%
Percentage of customers preferring Full Albums: 0.00%
